In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timezone

In [38]:
df=pd.read_excel("youtube_Trending_videos.xlsx")

In [39]:
df.head(2)

,id,Date-time,duration,channel_id,channel_title,title,category_id,live_broad_cast,Audio,view,like,commment,tag,country,country_iso,link
0,EdxguQAXeoU,2025-06-11T04:40:11Z,PT3M36S,UC9ChdqQRCaZmTCwSJ49tcbw,Sidhu Moose Wala,TAKE NOTES ( Official Audio) Sidhu Moose Wala ...,Music,none,English,7779282.0,1041330.0,156844.0,sidhu moosewala,United Arab Emirates,AE,https://www.youtube.com/watch?v=EdxguQAXeoU
1,uKAPYLvDJFQ,2025-06-11T04:30:23Z,PT2M10S,UC9ChdqQRCaZmTCwSJ49tcbw,Sidhu Moose Wala,0008 (Official Audio) Sidhu Moose Wala | Lat...,Music,none,English,6081458.0,811553.0,113710.0,sidhu moosewala,United Arab Emirates,AE,https://www.youtube.com/watch?v=uKAPYLvDJFQ


In [40]:
df.shape

(5500, 16)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5500 entries, 0 to 5499
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               5500 non-null   object 
 1   Date-time        5500 non-null   object 
 2   duration         5500 non-null   object 
 3   channel_id       5500 non-null   object 
 4   channel_title    5500 non-null   object 
 5   title            5500 non-null   object 
 6   category_id      5500 non-null   object 
 7   live_broad_cast  5500 non-null   object 
 8   Audio            3390 non-null   object 
 9   view             5499 non-null   float64
 10  like             5422 non-null   float64
 11  commment         5447 non-null   float64
 12  tag              3499 non-null   object 
 13  country          5500 non-null   object 
 14  country_iso      5500 non-null   object 
 15  link             5500 non-null   object 
dtypes: float64(3), object(13)
memory usage: 687.6+ KB


In [42]:
df.rename(columns={"category_id":"category","Audio":"language","commment":"comments"},inplace=True)

In [43]:
# which category has highest missing value in language
df["language"].fillna("Unknown",inplace=True)

C:\Users\dell\AppData\Local\Temp\ipykernel_16388\750579828.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["language"].fillna("Unknown",inplace=True)


In [44]:
df1=df[df["language"]=="Unknown"]
df1["counts"]=1
df1.shape

C:\Users\dell\AppData\Local\Temp\ipykernel_16388\3380810205.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["counts"]=1


(2110, 17)

In [45]:
df1.groupby(["category","language"])["counts"].sum()


category              language
Autos & Vehicles      Unknown       8
Comedy                Unknown     132
Education             Unknown      11
Entertainment         Unknown     411
Film & Animation      Unknown     100
Gaming                Unknown     192
Howto & Style         Unknown      54
Music                 Unknown     223
News & Politics       Unknown      39
People & Blogs        Unknown     683
Science & Technology  Unknown      25
Sports                Unknown     210
Travel & Events       Unknown      22
Name: counts, dtype: int64

In [46]:
df["Date-time"]=pd.to_datetime(df["Date-time"],utc=True)
df["totaldays_since_uploaded"]=round((datetime.now(timezone.utc)-df["Date-time"])/pd.Timedelta(days=1),2)

In [11]:
pip install pycountry pytz

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.




[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from pytz import country_timezones

In [48]:
country_timezones_map={code:timezones[0] for code , timezones in country_timezones.items() if timezones}
df["time_zone"]=df["country_iso"].map(country_timezones_map)
def convert_to_local_timezone(data):
    if data["time_zone"]:
        return data["Date-time"].tz_convert(data["time_zone"])
    return data["time_zone"]
df["local_datetime"]=df.apply(convert_to_local_timezone,axis=1)

In [49]:
df["local_date"]=df["local_datetime"].apply(lambda x:x.date())
df["local_time"]=df["local_datetime"].apply(lambda x:x.time())

In [57]:
pip install isodate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [50]:
from isodate import parse_duration

In [51]:
df["duration"]=df["duration"].apply(parse_duration)

In [52]:
df["duration"]=df["duration"].apply(lambda x: x.seconds/60)

In [58]:
df["duration"]=df["duration"].round(2)

In [ ]:
df["utc_date"]=df["Date-time"].dt.date
df["utc_time"]=df["Date-time"].dt.time

In [54]:
df.drop(columns=["Date-time","live_broad_cast","tag","country_iso","time_zone","local_datetime"],inplace=True)

In [55]:
df.dropna(subset=["like","comments"],inplace=True)
df.shape

(5372, 17)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5372 entries, 0 to 5499
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        5372 non-null   object 
 1   duration                  5372 non-null   float64
 2   channel_id                5372 non-null   object 
 3   channel_title             5372 non-null   object 
 4   title                     5372 non-null   object 
 5   category                  5372 non-null   object 
 6   language                  5372 non-null   object 
 7   view                      5372 non-null   float64
 8   like                      5372 non-null   float64
 9   comments                  5372 non-null   float64
 10  country                   5372 non-null   object 
 11  link                      5372 non-null   object 
 12  totaldays_since_uploaded  5372 non-null   float64
 13  local_date                5372 non-null   object 
 14  local_time   

In [61]:
df.head(2)

,id,duration,channel_id,channel_title,title,category,language,view,like,comments,country,link,totaldays_since_uploaded,local_date,local_time,utc_date,utc_time
0,EdxguQAXeoU,3.60,UC9ChdqQRCaZmTCwSJ49tcbw,Sidhu Moose Wala,TAKE NOTES ( Official Audio) Sidhu Moose Wala ...,Music,English,7779282.0,1041330.0,156844.0,United Arab Emirates,https://www.youtube.com/watch?v=EdxguQAXeoU,1.01,2025-06-11,08:40:11,2025-06-11,04:40:11
1,uKAPYLvDJFQ,2.17,UC9ChdqQRCaZmTCwSJ49tcbw,Sidhu Moose Wala,0008 (Official Audio) Sidhu Moose Wala | Lat...,Music,English,6081458.0,811553.0,113710.0,United Arab Emirates,https://www.youtube.com/watch?v=uKAPYLvDJFQ,1.02,2025-06-11,08:30:23,2025-06-11,04:30:23


In [23]:
df.describe()

,duration,view,like,comments,totaldays_since_uploaded
count,5372.000000,5.372000e+03,5.372000e+03,5372.000000,5372.000000
mean,16.540876,1.641253e+07,3.996029e+05,5967.833023,6.971828
std,44.220853,2.995978e+07,7.383171e+05,19134.419003,5.361060
min,0.166667,2.657200e+04,1.060000e+02,0.000000,0.304671
25%,0.500000,6.603230e+05,1.880875e+04,494.000000,2.727961
50%,2.533333,2.909099e+06,9.269700e+04,1366.500000,5.886094
75%,15.633333,1.841674e+07,4.434440e+05,3656.000000,9.687480
max,714.966667,2.329878e+08,6.085883e+06,496876.000000,36.626349


In [62]:
df.to_csv("youtube_trending_videos1.csv",index=False)

In [65]:
data=pd.read_csv("youtube_trending_videos1.csv")
data.drop_duplicates(subset="id",inplace=True)
data.to_csv("trending_videos.csv",index=False)

In [27]:
pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/45.0 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.0 kB 682.7 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 495.5 kB/s eta 0:00:01
   ------------------------------------ --- 41.0/45.0 kB 495.5 kB/s eta 0:00:01
   ---------------------------------------- 45.0/45.0 kB 222.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 660.6 kB/s eta 0:00:04
   - -------------------------------------- 0.1/2.1 MB 871.5 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 804.6 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 853.3 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 853.3 kB/s eta 0:00:03
   -- ------------------------------------- 0.1/2.1 MB 853.3 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/2.1 MB 811.7 kB/s eta 0:00:03
   ------ --------------------------------- 0.3/2.1 MB 846.5 kB/s eta 0:00:03
   ------- -------------------------------- 0.4/2.1 MB 958.4 kB/s eta 0:00:02
   -------- ------------------------------- 0.5/2.1 MB 994.6 kB/s eta 0:00:02
   -


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [66]:
import pymysql
from sqlalchemy import create_engine

In [67]:
my_engine=create_engine("mysql+pymysql://root@localhost:3306/youtube")
try:
    my_engine
    print("connected successfully")
except:
    print("can't connect")

connected successfully


In [68]:
df.to_sql(name="Youtube_trending_videos",con=my_engine,if_exists="append",index=False)

C:\Users\dell\AppData\Local\Temp\ipykernel_16388\1594808472.py:1: UserWarning: The provided table name 'Youtube_trending_videos' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  df.to_sql(name="Youtube_trending_videos",con=my_engine,if_exists="append",index=False)


5372

In [69]:
data.to_sql(name="trending_videos",con=my_engine,if_exists="append",index=False)

1961